Now, we start to do some real processing.

<b>Note:</b>
<br>
<br>
Some changes are required depending on the model you intend to use for prediction:

To change the model's configuration, simply replace the existing config parameter with the desired one in the line of the code below:

```python 
   cfg = parse_adaptor_jpnb(config_class=ProgramConfig, config="./semantic_seg_2d_inference_2class.yaml")
```

 For the retarining of the existing model with improvements no changes are needed here:

```python
   cfg = parse_adaptor_jpnb(config_class=ProgramConfig, config="./semantic_seg_2d_inference_2class.yaml") 
```
   
 For the Probabilistic model use:  
 
```python
   cfg = parse_adaptor_jpnb(config_class=ProgramConfig, config="./probabilistic_semantic_seg2D_inference_2Class.yaml")  
```
 
Depending on your model selection, you'll need to choose the correct weights.<br>
This means changing the input in the following line of the code below:

```python 
   cfg.model.checkpoint = Path("./version_2023_06.ckpt")
```

If you desire to test the retraining of the last model, you need to use: 

```python 
   cfg.model.checkpoint = Path("./ClassicUnet_16_06_2025.ckpt")
```

If you desire to test the probabilistic model you need to use:

```python 
   cfg.model.checkpoint = Path("./ProbUnet_16_06_2025.ckpt")
```

These .ckpt files are now available via the same download [LINK](https://ambiomcloud.isas.de/index.php/s/CwcfFRt8eQ9gKWj).

In [1]:
from core.inference import create_inference_menu
create_inference_menu()

In the future `np.bool` will be defined as the corresponding NumPy scalar.
In the future `np.bool` will be defined as the corresponding NumPy scalar.
In the future `np.bool` will be defined as the corresponding NumPy scalar.


Text(value='./semantic_seg_2d_inference_2class.yaml', description='YAML Path:', layout=Layout(width='80%'), pl…

Text(value='./version_2023_06.ckpt', description='CKPT Path:', layout=Layout(width='80%'), placeholder='Weight…

Text(value='/path/to/splitted/3d/files', description='Input base path:', layout=Layout(width='80%'), placehold…

Button(button_style='success', description='Run Inference', style=ButtonStyle())

Output()